In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

from src.ad_hmm import sku_predict
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 2

### Create 1/2 sku dataset to import in matlab and forecast

In [4]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

(846632, 15)


In [6]:
#clean train/test
train = train_test[(train_test.validation==False) & (train_test.validation_clean==True)]
validation = train_test[(train_test.validation==True) & (train_test.validation_clean==True)]
train.shape,validation.shape

((797490, 15), (3815, 15))

In [7]:
sku_id = 479
train_sku = train.loc[train.Encoded_SKU_ID == sku_id]
train_sku.columns

Index(['Encoded_SKU_ID', 'SALES_DATE', 'SUBCLASS_NAME', 'CLASS_NAME',
       'ML_NAME', 'CATEGORY_NAME', 'RETAIL_PRICE', 'PROMO_PRICE',
       'COMPETITOR_PRICE', 'Inventory', 'DAILY_UNITS', 'max_SALES_DATE',
       'delta_days', 'validation', 'validation_clean'],
      dtype='object')

In [34]:
train_sku["if_promo"] = (~train_sku["PROMO_PRICE"].isin(["?"])).astype("int")
train_sku["if_competitor"] = (~train_sku["COMPETITOR_PRICE"].isin(["?"])).astype("int")

train_sku["min_sales_price"] = pd.to_numeric(np.where(train_sku.if_promo>0, train_sku.PROMO_PRICE, train_sku.RETAIL_PRICE))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/3526041755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["if_promo"] = (~train_sku["PROMO_PRICE"].isin(["?"])).astype("int")
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/3526041755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["if_competitor"] = (~train_sku["COMPETITOR_PRICE"].isin(["?"])).astype("int")
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/3526041755.py:4: SettingW

In [35]:
train_sku.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean,if_promo,if_competitor,min_sales_price,comp_price_clean
1275,479,2020-02-03,DISHWASHER,DISHWASHERS,ML - Kitchen,VP - Major Appliances,944.99,?,939,Fully-Stocked,1,2022-07-31,909,False,True,0,1,944.99,939.0
2883,479,2019-11-10,DISHWASHER,DISHWASHERS,ML - Kitchen,VP - Major Appliances,944.99,649.99,?,Constrained,50,2022-07-31,994,False,True,1,0,649.99,NaN
4322,479,2020-03-04,DISHWASHER,DISHWASHERS,ML - Kitchen,VP - Major Appliances,944.99,?,939,Fully-Stocked,3,2022-07-31,879,False,True,0,1,944.99,939.0
5403,479,2020-09-18,DISHWASHER,DISHWASHERS,ML - Kitchen,VP - Major Appliances,944.99,?,943,Fully-Stocked,9,2022-07-31,681,False,True,0,1,944.99,943.0
6930,479,2021-01-12,DISHWASHER,DISHWASHERS,ML - Kitchen,VP - Major Appliances,944.99,749.99,748,Moderate,16,2022-07-31,565,False,True,1,1,749.99,748.0


In [36]:
train_sku["comp_price_clean"] = pd.to_numeric(np.where(train_sku.if_competitor>0,train_sku.COMPETITOR_PRICE, np.nan))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/1354638414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["comp_price_clean"] = pd.to_numeric(np.where(train_sku.if_competitor>0,train_sku.COMPETITOR_PRICE, np.nan))


In [37]:
train_sku["price_delta"] = np.where(train_sku.if_competitor>0, train_sku.min_sales_price - train_sku.comp_price_clean,\
                                   -train_sku.min_sales_price)

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/4294698538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["price_delta"] = np.where(train_sku.if_competitor>0, train_sku.min_sales_price - train_sku.comp_price_clean,\


In [49]:
train_sku[["year","month","date"]] = pd.DataFrame(train_sku["SALES_DATE"].astype("str").apply(lambda x: x.split("-")).tolist(),\
            index = train_sku.index)

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/4033675466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku[["year","month","date"]] = pd.DataFrame(train_sku["SALES_DATE"].astype("str").apply(lambda x: x.split("-")).tolist(),\
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_60533/4033675466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku[["year","month","date"]] = pd.DataFrame(train_sku["SALES_DATE"].astype("str").apply(lambda x: x.split("-")).tolist(),

In [53]:
train_sku[["year","month","date","if_promo","if_competitor","min_sales_price","price_delta","DAILY_UNITS"\
          ]].to_csv(f"../../data/markov_regression_train_{sku_id}.csv",index=False)

In [52]:
train_sku[["year","month","date","if_promo","if_competitor","min_sales_price","price_delta","DAILY_UNITS"\
          ]].head()

,year,month,date,if_promo,if_competitor,min_sales_price,price_delta,DAILY_UNITS
1275,2020,02,03,0,1,944.99,5.99,1
2883,2019,11,10,1,0,649.99,-649.99,50
4322,2020,03,04,0,1,944.99,5.99,3
5403,2020,09,18,0,1,944.99,1.99,9
6930,2021,01,12,1,1,749.99,1.99,16


In [40]:
train_sku.shape

(1065, 20)